## Etiquetado de tweets

In [70]:
import pandas as pd
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [71]:
system_prompt = """You are a data scientist labelling tweets. Given a tweet, classify it into one of the following categories:

Tesla Products
Customer Experience
Performance & Innovation
Financial News
Environmental Impact
Industry News
Charging Infrastructure

If the tweet does not clearly fit into any of these categories or is not relevant, classify it as "Not relevant".

Example:

Tweet:

Tesla's new Model S Plaid is the fastest production car ever made!

Category:

Tesla Products

Example:

Tweet:

I'm having a terrible time with Tesla customer service. They've been ignoring my emails for weeks.

Category:

Customer Experience

I want you to be very rigorous with the labelling. You have to respond with the following phrase: Category: [response]"""

In [72]:
df = pd.read_csv('../../data/clean/tesla_tweets_clean.csv')
test_df = df.sample(10)
df.head()

,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes ...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love in my MariaCallas I KNOW Y art HOLDonLIN...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love in my MariaCallas I KNOW Y art HOLDonLIN...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Large...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,few chargingstations in my area I dont have a...


In [75]:
df_labelled = pd.DataFrame(columns=['Link', 'Date', 'Text', 'Category'])

for index, row in test_df.iterrows():
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row['Text']}
      ],
        max_tokens=10,
        temperature=0.9,
    )
    answer = completion.choices[0].message['content'].replace('Category: ', '')

Category: Not relevant
Category: Not relevant
Category: Not relevant
Category: Financial News
Category: Industry News
Category: Not relevant
Category: Financial News
Category: Tesla Products
Category: Not relevant
Category: Tesla Products
